# Assignment for Coursera_Capstone

### Import Necessary libraries

In [52]:
## import necessary libraries
import pandas as pd
import numpy as np
import sklearn 
import matplotlib as mpl
import requests
from bs4 import BeautifulSoup

### Scrape datasets from online
We need to present the datasets in the format of dataframe for further operations

In [45]:
## scrape datasets from website 
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find_all('table')[0]
Toronto_data_raw = pd.read_html(str(table))
Toronto_data1 = Toronto_data_raw[0]
Toronto_data1.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


### Set the column names

In [51]:
## clean the dataset and set it into a dataframe format
Toronto_data = Toronto_data1.drop([0],axis = 0)
Toronto_data = Toronto_data.reset_index()
Toronto_data.columns = ['index','Postcode', 'Borough','Neighborhood']
Toronto_data = Toronto_data.drop(['index'], axis = 1)
Toronto_data.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Clean data with unassigned borough values

In [63]:
## Ignore rows with unassigned borough values 
Toronto_data = Toronto_data[Toronto_data['Borough'] != 'Not assigned']

Toronto_data.shape

(211, 3)

### Format data with same postcode but different neighborhoods

In [68]:
## use another dataframe for the operations
Toronto_try = Toronto_data

In [71]:
## Test if there are repeating rows in the dataframe
for i in range(211):
    for j in range(211):
        if i != j and Toronto_try.iloc[i]['Postcode'] ==  Toronto_try.iloc[j]['Postcode'] and Toronto_try.iloc[i]['Neighborhood'] == Toronto_try.iloc[j]['Neighborhood']:
            print (i,j)

In [93]:
## combine the neighborhoods for the same postcode value
for i in range(1, 211):
    if Toronto_try.iloc[i]['Postcode'] ==  Toronto_try.iloc[i-1]['Postcode'] and Toronto_try.iloc[i]['Neighborhood'] != Toronto_try.iloc[i-1]['Neighborhood']:
        Toronto_try.iloc[i]['Neighborhood'] = str(Toronto_try.iloc[i-1]['Neighborhood']) + ', ' + Toronto_try.iloc[i]['Neighborhood'] 

## reset index 
Toronto_try1 = Toronto_try.reset_index()
Toronto_try1 = Toronto_try1.drop(['index'], axis = 1)

## drop redundant rows
redundant_rows = []
for i in range(1, 211):
    if Toronto_try1.iloc[i]['Postcode'] == Toronto_try1.iloc[i-1]['Postcode']:
        redundant_rows.append(i-1) 

## reset index for the new dataframe again
Toronto_try2 = Toronto_try1.drop(redundant_rows)
Toronto_try2 = Toronto_try2.reset_index()  
Toronto_try2 = Toronto_try2.drop(['index'], axis = 1)
Toronto_try2.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Harbourfront, Harbourfront, Harb..."
3,M6A,North York,"Lawrence Heights, Lawrence Heights, Lawrence H..."
4,M7A,Queen's Park,Not assigned


### Format missing data of neighborhood with borough value

In [97]:
## use row iterations to fill in the missing data of neighborhood
for index, row in Toronto_try2.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
## use another explainative dataframe name to represent the final data 
Toronto_final = Toronto_try2
Toronto_final.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Harbourfront, Harbourfront, Harb..."
3,M6A,North York,"Lawrence Heights, Lawrence Heights, Lawrence H..."
4,M7A,Queen's Park,Queen's Park


### Shape of the dataframe 

In [99]:
Toronto_final.shape

(103, 3)